In [ ]:
# !pip install pandas_datareader keras seaborn
# !conda install -y -c conda-forge fbprophet
# !pip install pydot graphviz
import boto3
import base64
from botocore.exceptions import ClientError
from IPython.display import display
import pandas_datareader
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense, LSTM, InputLayer, Attention
import seaborn as sns
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.callbacks import EarlyStopping

In [ ]:
tickers = ['AAPL']
metric = 'low'
pc_metric = f'{metric}_percent_change'
norm_metric = f'{pc_metric}_norm'
lookback=100
def get_secret():
    secret_name = "alpha_vantage"
    region_name = "us-east-2"
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        display(e)
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            secret = base64.b64decode(get_secret_value_response['SecretBinary'])
    return secret   
def format_dates(daily_stocks_data):
    df = daily_stocks_data.copy() 
    df['date']=df.index
    df.reset_index(inplace=True, drop=True)
    return df
def add_percent_change(daily_stocks_data, metric):
    percents = list()
    for index, row in daily_stocks_data.iterrows():
        old = row[metric]
        try:
            new = daily_stocks_data.iloc[index + 1][metric]
        except Exception as e:
            percents.append(np.nan) ## no next value, so this is undefined
            continue
        percents.append((new-old)/new)
    cp_df = daily_stocks_data.copy()
    cp_df[f'{metric}_percent_change']=percents
    return cp_df
def add_norm(df, label):
    arr = np.array([x*1000 for x in df[label].to_numpy()]).reshape(-1, 1)
#     norm = normalize(arr, norm='l1')
    norm = arr
    new_df = df.copy()
    new_df[f'{label}_norm'] = norm
    return new_df
def to_ts_df(daily_stocks_data, lookback, metric):
    ## column names
    columns = list()
    for i in range(lookback):
        columns.append(f'{metric}_{i}')
    columns.append(f'{metric}_target')
    df = pd.DataFrame(columns=columns)
    ## columns
    data = daily_stocks_data[metric].to_numpy()
    for index, col in enumerate(df.columns):
        df[col] = data[index:len(data)-lookback+index]
    ## dates index
    dates = daily_stocks_data.date.to_numpy()[:-lookback]
    df.insert(0, 'date', dates)
    return df
def to_ts(ts_df):
    data = list()
    targets = list()
    for index, row in ts_df.iloc[:,1:].iterrows():
        rnp = row.to_numpy()
        data.append([[x] for x in rnp[:-1]])
        targets.append(rnp[-1])
    data = np.array(data)
    targets = np.array(targets)
    return data, targets

In [ ]:
ALPHA_API_KEY = get_secret()

In [ ]:
# daily_stocks_data_raw = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=tickers, api_key=ALPHA_API_KEY, function='TIME_SERIES_DAILY').read()
daily_stocks_data = format_dates(daily_stocks_data_raw) 
daily_stocks_data = add_percent_change(daily_stocks_data, metric)
daily_stocks_data[daily_stocks_data[pc_metric].isnull()] = 0
daily_stocks_data = add_norm(daily_stocks_data, pc_metric)
ts_df = to_ts_df(daily_stocks_data, lookback, pc_metric)
data, targets = to_ts(ts_df)
display(daily_stocks_data)
display(ts_df)

In [ ]:
def deep_lstm():
    model = Sequential()
    model.add(InputLayer(input_shape=(None,1)))
#     model.add(LSTM(12, return_sequences=True))
#     model.add(LSTM(12, return_sequences=True))
#     model.add(LSTM(6, return_sequences=True))
    model.add(LSTM(6, return_sequences=True))
    model.add(LSTM(2, return_sequences=True))
    model.add(LSTM(1))
    model.add(Dense(1))
    model.compile(loss='mae', metrics=['mse','mape'])
    return model

In [40]:
model = deep_lstm()
model.summary()
# plot_model(model)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, None, 2)           32        
_________________________________________________________________
lstm_17 (LSTM)               (None, None, 2)           40        
_________________________________________________________________
lstm_18 (LSTM)               (None, None, 2)           40        
_________________________________________________________________
lstm_19 (LSTM)               (None, None, 2)           40        
_________________________________________________________________
lstm_20 (LSTM)               (None, None, 2)           40        
_________________________________________________________________
lstm_21 (LSTM)               (None, 1)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                

In [ ]:
early = EarlyStopping(patience=4, restore_best_weights=True)
model.fit(x=data, y=targets, batch_size=36, validation_split=0.2, epochs=100, callbacks=[early])

Epoch 1/100
 58/110 [==============>...............] - ETA: 15s - loss: 0.0177 - mse: 0.0179 - mape: 9661.1162